# Images, Darks, & Flats with EPICS area detector, ophyd, and Bluesky

**TODO**: work-in-progress

The HDF5 File Writer
[plugin](https://areadetector.github.io/areaDetector/ADCore/NDFileHDF5.html) can
be configured to save image frames into separate datasets within the same HDF5
data file.  The selection of frame type (image frame, background/dark frame,
white/flat frame) is made by use of an existing PV in area detector:
`$(P):cam1:FrameType` which is an *mbbo* record.  In ophyd, the readback version
of this PV: `$(P):cam1:FrameType_RBV` is used to define operational values for
the ophyd device.  Be sure to configure the readback PV with the same values.

meaning | PV value | PV field | default | dxchange | NeXus
--- | --- | --- | --- | --- | ---
image frame | 0 | `ZRST` | `Normal` | `/exchange/data` | `/entry/data/data`
background/dark frame | 1 | `ONST` | `Background` | `/exchange/data_dark` | `/entry/data/dark`
white/flat frame | 2 | `TWST` | `FlatField` | `/exchange/data_white` | `/entry/data/white`
not used | 3 | `THST` | `DblCorrelation` | `""` | `""`
not used | 4 | `FRST` | `""` | `""` | `""`

Using the values (addresses of HDF5 groups) for the chosen format (dxchange or
NeXus), set the fields of **both** these PVs (`$(P):cam1:FrameType` and
`$(P):cam1:FrameType_RBV`).

In NeXus, the detector data is stored within the instrument group, traditionally
at `/entry/instrument/detector/`.  We then hard link that into the `/entry/data`
group.  The hard link allows us to shorten the address string to fit within the
maximum 25 characters allowed by the EPICS
[mbbo](https://epics.anl.gov/base/R7-0/6-docs/mbboRecord.html) record.  We write
the data into the `/entry/data` group and hard link it (in the layout.xml file)
to the instrument group.

**Tip**:  Whichever format you use, be sure the fields are put in the IOC's
autosave configuration so they are restored when the IOC is restarted!

The area detector attributes XML file needs the selection PV included in its
list.  We'll call it `SaveDest` so we can use the same name in the layout file:

    <Attribute 
        name="SaveDest"
        type="EPICS_PV"
        source="13SIM1:cam1:FrameType"
        dbrtype="DBR_STRING"
        description="image, dark, or flat frame"/>

Then, the HDF5 layout XML file refers to this `SaveDest` attribute in the setup
(add this to the XML file just after the opening `hdf5_layout` and before the
first `group` element)

    <global name="detector_data_destination" ndattribute="SaveDest" />

The name `detector_data_destination` is hard-coded in the source code of the HDF5 file writer.

In the Bluesky plan, write the frame type with 0: image, 1: dark, 2: flat before
acquiring the image frame of that type.  Then the HDF5 file writer will direct
the image frame to the correct dataset as specified by the ZRST, ONST, or TWST
field, respectively.

## Try it with a NeXus file and the AD SimDetector

First, configure an instance of the sim detector.

In [1]:
import pathlib
import h5py

import apstools
import databroker
from apstools.devices import CamMixin_V34
from ophyd import Component
from ophyd import Device
from ophyd import EpicsSignal
from ophyd import SimDetector
from ophyd.areadetector import SimDetectorCam
from ophyd.areadetector.plugins import HDF5Plugin_V34
from ophyd.areadetector.plugins import ImagePlugin_V34
from ophyd.areadetector.plugins import PvaPlugin_V34

# IOC sees /tmp directory, here it is on host workstation.
WORKSTATION_PREFIX = pathlib.Path("/tmp/docker_ioc/iocad/tmp")

class MySimDetectorCam(CamMixin_V34, SimDetectorCam):
    """Cam support for latest AD changes."""

class MyHDF5Plugin(HDF5Plugin_V34):
    layout_filename = Component(EpicsSignal, "XMLFileName", kind="config", string=True)
    layout_filename_valid = Component(EpicsSignal, "XMLValid_RBV", string=True)
    nd_attr_status = Component(EpicsSignal, "NDAttributesStatus", string=True)

class MyDetector(SimDetector):
    cam = Component(MySimDetectorCam, suffix="cam1:")
    hdf1 = Component(MyHDF5Plugin, suffix="HDF1:")
    image = Component(ImagePlugin_V34, suffix="image1:")
    pva1 = Component(PvaPlugin_V34, suffix="Pva1:")

In [2]:
ad_prefix = "ad:"
adsimdet = MyDetector(ad_prefix, name="det")
adsimdet.wait_for_connection()

Setup Bluesky

In [3]:
import bluesky
from bluesky import plans as bp

cat = databroker.temp().v2
RE = bluesky.RunEngine()
RE.subscribe(cat.v1.insert)

RE.md["title"] = "images, darks, & flats"
RE.md["versions"]["apstools"] = apstools.__version__
RE.md["repository"] = "bluesky_training"
RE.md["notebook"] = "images_darks_flats"

Set the counting time per frame (something short).

In [4]:
def set_ad_count_time(exposure=1, period=1):
    adsimdet.cam.stage_sigs["acquire_time"] = exposure
    adsimdet.cam.stage_sigs["acquire_period"] = period

set_ad_count_time(exposure=0.02, period=0.1)

Set other detector parameters.

In [5]:
def ad_setup(det=adsimdet, nframes=1):
    """Make this a function.  We'll use again later."""
    det.cam.acquire.put(0)
    det.cam.frame_type.kind = "config"
    det.hdf1.capture.put(0)
    det.hdf1.compression.put("zlib")  # better than `"None"` (default)
    det.hdf1.create_directory.put(-5)
    det.hdf1.file_name.put("test_image")
    det.hdf1.file_path.put("/tmp")
    det.hdf1.kind = 3  # config | normal

    if "compression" in det.hdf1.stage_sigs:
        det.hdf1.stage_sigs.pop("create_directory")

    # The plugins do not block, the cam must wait for the plugins to finish.
    for nm in det.component_names:
        obj = getattr(det, nm)
        if "blocking_callbacks" in dir(obj):  # is it a plugin?
            obj.stage_sigs["blocking_callbacks"] = "No"
    det.cam.stage_sigs["wait_for_plugins"] = "Yes"

    det.cam.stage_sigs["num_images"] = nframes
    det.hdf1.stage_sigs["num_capture"] = 0  # capture ALL frames received
    det.hdf1.stage_sigs["auto_increment"] = "Yes"
    det.hdf1.stage_sigs["auto_save"] = "Yes"
    det.hdf1.stage_sigs["file_template"] = "%s%s_%4.4d.h5"
    det.hdf1.stage_sigs["file_write_mode"] = "Stream"
    det.hdf1.stage_sigs["store_attr"] = "Yes"  # need in this notebook
    det.hdf1.stage_sigs["store_perform"] = "No"  # optional
    det.hdf1.stage_sigs["auto_increment"] = "Yes"
    det.hdf1.stage_sigs["capture"] = 1  # ALWAYS last
    det.hdf1.stage_sigs.move_to_end("capture")  # ... just in case

NUM_FRAMES = 5
ad_setup(det=adsimdet, nframes=NUM_FRAMES)

In [6]:
# Clear these settings __for this demo__.
adsimdet.hdf1.layout_filename.put("")
adsimdet.hdf1.nd_attributes_file.put("")

In [7]:
def check_adplugin_primed(plugin, allow_priming=True):
    from apstools.devices import AD_plugin_primed
    from apstools.devices import AD_prime_plugin2

    # this step is needed for ophyd
    if not AD_plugin_primed(plugin):
        if allow_priming:
            print(f"Priming {plugin.dotted_name}")
            AD_prime_plugin2(plugin)
        else:
            raise RuntimeError(
                f"Detector plugin '{plugin.dotted_name}' must be primed first."
            )

check_adplugin_primed(adsimdet.hdf1)

Priming hdf1


In [8]:
def dict_to_table(d, printing=True):
    import pyRestTable

    if len(d) == 0:
        return

    table = pyRestTable.Table()
    table.labels = "key value".split()
    table.rows = [[k, v] for k, v in d.items()]

    if printing:
        print(table)
    else:
        return table


def readings_to_table(d, printing=True):
    import pyRestTable

    if len(d) == 0:
        return

    labels = sorted(set([k for v in d.values() for k in v.keys()]))
    table = pyRestTable.Table()
    # fmt: off
    table.labels = [ "name", ] + list(labels)
    for k, reading in d.items():
        row = [k, ] + [reading.get(r, "") for r in labels]
        table.addRow(row)
    # fmt: on

    if printing:
        print(table)
    else:
        return table


def print_overview(device):
    cfg = device.describe_configuration()
    for k, readings in device.read_configuration().items():
        if k not in cfg:
            cfg[k] = readings
        else:
            cfg[k].update(readings)

    if len(cfg) > 0:
        print(f"'{device.name}' configuration:")
    readings_to_table(cfg)


def practice_device_staging(device):
    print(f"Before staging '{device.name}'")
    device.stage()
    print(f"Device '{device.name}' staged")
    device.unstage()
    print(f"Device '{device.name}' unstaged")

In [9]:
print("RunEngine metadata")
dict_to_table(RE.md)

RunEngine metadata
========== =============================================================
key        value                                                        
========== =============================================================
versions   {'ophyd': '1.7.0', 'bluesky': '1.10.0', 'apstools': '1.6.14'}
title      images, darks, & flats                                       
repository bluesky_training                                             
notebook   images_darks_flats                                           
========== =============================================================



In [10]:
print("'adsimdet.stage_sigs' stage_sigs")
dict_to_table(adsimdet.stage_sigs)

'adsimdet.stage_sigs' stage_sigs


In [11]:
print("'adsimdet.cam.stage_sigs' stage_sigs")
dict_to_table(adsimdet.cam.stage_sigs)

'adsimdet.cam.stage_sigs' stage_sigs
================ =====
key              value
================ =====
acquire_time     0.02 
acquire_period   0.1  
wait_for_plugins Yes  
num_images       5    
================ =====



In [12]:
print("'adsimdet.hdf1.stage_sigs' stage_sigs")
dict_to_table(adsimdet.hdf1.stage_sigs)

'adsimdet.hdf1.stage_sigs' stage_sigs
========================== =============
key                        value        
========================== =============
enable                     1            
blocking_callbacks         No           
parent.cam.array_callbacks 1            
num_capture                0            
auto_increment             Yes          
auto_save                  Yes          
file_template              %s%s_%4.4d.h5
file_write_mode            Stream       
store_attr                 Yes          
store_perform              No           
capture                    1            
========================== =============



In [13]:
print_overview(adsimdet)

'det' configuration:
====================== ======= ============================================================ ================ ========= ===== ============================ ================= ===== ================ ==================
name                   dtype   enum_strs                                                    lower_ctrl_limit precision shape source                       timestamp         units upper_ctrl_limit value             
====================== ======= ============================================================ ================ ========= ===== ============================ ================= ===== ================ ==================
det_cam_acquire_period number                                                               0.0              3         []    PV:ad:cam1:AcquirePeriod_RBV 1679532683.401695       0.0              0.005             
det_cam_acquire_time   number                                                               0.0              3         []  

Now, count `adsimdet` using default settings.

In [14]:
RE(bp.count([adsimdet]))

('77279505-7a56-4056-88b5-a6c1888d5e9b',)

In [15]:
def local_h5_file(det):
    path = WORKSTATION_PREFIX.parent
    ioc_file = det.hdf1.full_file_name.get()
    return path / ioc_file.lstrip("/")

def check_h5_file(det):
    hfile = local_h5_file(det)
    print(f"{hfile.exists()=} {hfile.name=}")
    addr = "/entry/instrument/NDAttributes"
    with h5py.File(hfile, "r") as root:
        group = root[addr]
        print(f"{len(group)=}  {group=}")
        print(f"members: {[k for k in group]}")

check_h5_file(adsimdet)

hfile.exists()=True hfile.name='test_image_0000.h5'
len(group)=4  group=<HDF5 group "/entry/instrument/NDAttributes" (4 members)>
members: ['NDArrayEpicsTSSec', 'NDArrayEpicsTSnSec', 'NDArrayTimeStamp', 'NDArrayUniqueId']


Note that we see 4 attributes.

In [16]:
print(f"{adsimdet.hdf1.nd_attributes_file.get() = }")

adsimdet.hdf1.nd_attributes_file.get() = ''


In [17]:
print(
    f"{adsimdet.cam.frame_type.pvname=}"
    f" value={adsimdet.cam.frame_type.get()}"
    f" choices={adsimdet.cam.frame_type.enum_strs}"
)

adsimdet.cam.frame_type.pvname='ad:cam1:FrameType_RBV' value=0 choices=('/entry/data/data', '/entry/data/dark', '/entry/data/flat')


We'll make a custom attributes file with the `FrameType` PV with the attributes
name `SaveDest` and then load that file into the EPICS AD.

In [18]:
xml_file = WORKSTATION_PREFIX / "attributes.xml"  # IOC sees: /tmp/attributes.xml
XML_ATTRIBUTES = f"""
<?xml version="1.0" standalone="no" ?>
<Attributes
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:noNamespaceSchemaLocation="../../../../../ADCore/XML_schema/NDAttributes.xsd"
    >
    <Attribute name="AcquireTime"         type="EPICS_PV" source="{ad_prefix}cam1:AcquireTime"      dbrtype="DBR_NATIVE"  description="Camera acquire time"/>
    <Attribute name="SaveDest"            type="EPICS_PV" source="{adsimdet.cam.frame_type.pvname}"    dbrtype="DBR_STRING"  description="image, dark, or flat frame"/>
    <Attribute name="ImageCounter"        type="PARAM"    source="ARRAY_COUNTER"                datatype="INT"        description="Image counter"/>
    <Attribute name="MaxSizeX"            type="PARAM"    source="MAX_SIZE_X"                   datatype="INT"        description="Detector X size"/>
    <Attribute name="MaxSizeY"            type="PARAM"    source="MAX_SIZE_Y"                   datatype="INT"        description="Detector Y size"/>
    <Attribute name="CameraModel"         type="PARAM"    source="MODEL"                        datatype="STRING"     description="Camera model"/>
    <Attribute name="AttributesFileParam"  type="PARAM"   source="ND_ATTRIBUTES_FILE"            datatype="STRING"     description="Attributes file param"/>
    <Attribute name="CameraManufacturer"  type="PARAM"    source="MANUFACTURER"                 datatype="STRING"     description="Camera manufacturer"/>
</Attributes>
"""

print(f"{xml_file=}")
with open(xml_file, "w") as f:
    f.write(XML_ATTRIBUTES.strip())
print(f"{xml_file.exists()=} {xml_file=}")

xml_file=PosixPath('/tmp/docker_ioc/iocad/tmp/attributes.xml')
xml_file.exists()=True xml_file=PosixPath('/tmp/docker_ioc/iocad/tmp/attributes.xml')


In [19]:
adsimdet.hdf1.nd_attributes_file.put("/tmp/attributes.xml")

Check that this attributes XML file was read and that the IOC reports everything
was OK with it.  Fix any errors before proceeding.

In [20]:
print(f"{adsimdet.hdf1.nd_attr_status.get() = }")

adsimdet.hdf1.nd_attr_status.get() = 'File not found'


Count again and check if more attributes are in the HDF5 file.  Verify that
`SaveDest` is in the list of members.

In [21]:
RE(bp.count([adsimdet]))
check_h5_file(adsimdet)

hfile.exists()=True hfile.name='test_image_0001.h5'
len(group)=12  group=<HDF5 group "/entry/instrument/NDAttributes" (12 members)>
members: ['AcquireTime', 'AttributesFileParam', 'CameraManufacturer', 'CameraModel', 'ImageCounter', 'MaxSizeX', 'MaxSizeY', 'NDArrayEpicsTSSec', 'NDArrayEpicsTSnSec', 'NDArrayTimeStamp', 'NDArrayUniqueId', 'SaveDest']


We've been using the default layout, `hdf5_layout_nexus.xml`, from the IOC's
directory.  We'll need our own so that we can customize it.

Here, we create the new `/tmp/default_layout.xml` (using the XML content
supplied with the ADSimDetector) and tell the HDF5 plugin to use it.  Rather
than use an XML library to write this file, we'll create all the content here as
text and write the file with the usual text file tools. Then, we'll check that
it was read and the file content was acceptable:

In [22]:
xml_file = WORKSTATION_PREFIX / "default_layout.xml"  # IOC sees: /tmp/default_layout.xml
XML_LAYOUT = """
<?xml version="1.0" standalone="no" ?>
<hdf5_layout
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:noNamespaceSchemaLocation="../../../../../ADCore/XML_schema/hdf5_xml_layout_schema.xsd"
    >
  <group name="entry">
    <attribute name="NX_class" source="constant" value="NXentry" type="string" />
    <group name="instrument">
      <attribute name="NX_class" source="constant" value="NXinstrument" type="string" />
      <group name="detector">
        <attribute name="NX_class" source="constant" value="NXdetector" type="string" />
        <dataset name="data" source="detector" det_default="true">
          <attribute name="NX_class" source="constant" value="SDS" type="string" />
          <attribute name="signal" source="constant" value="1" type="int" />
          <attribute name="target" source="constant" value="/entry/instrument/detector/data" type="string" />
        </dataset>
        <group name="NDAttributes">
          <attribute name="NX_class" source="constant" value="NXcollection" type="string" />
          <dataset name="ColorMode" source="ndattribute" ndattribute="ColorMode">
          </dataset>
        </group>          <!-- end group NDAttribute -->
      </group>            <!-- end group detector -->
      <group name="NDAttributes" ndattr_default="true">
        <attribute name="NX_class" source="constant" value="NXcollection" type="string" />
      </group>            <!-- end group NDAttribute (default) -->
      <group name="performance">
        <dataset name="timestamp" source="ndattribute" />
      </group>            <!-- end group performance -->
    </group>              <!-- end group instrument -->
    <group name="data">
      <attribute name="NX_class" source="constant" value="NXdata" type="string" />
      <hardlink name="data" target="/entry/instrument/detector/data" />
    </group>              <!-- end group data -->
  </group>                <!-- end group entry -->
</hdf5_layout>
"""

with open(xml_file, "w") as f:
    f.write(XML_LAYOUT.strip())
print(f"{xml_file.exists()=} {xml_file=}")

xml_file.exists()=True xml_file=PosixPath('/tmp/docker_ioc/iocad/tmp/default_layout.xml')


Tell the IOC to use this new layout.

In [23]:
adsimdet.hdf1.layout_filename.put("/tmp/default_layout.xml")

Check that the IOC reports the file is found and its XML contents are valid.  The result should be `'Yes'`.

In [24]:
print(f"{adsimdet.hdf1.layout_filename_valid.get()=}")

adsimdet.hdf1.layout_filename_valid.get()='Yes'


Try it and verify `SaveDest` is present in the output file.  Result should be `True`.

In [25]:
RE(bp.count([adsimdet]))

h5_file = local_h5_file(adsimdet)
with h5py.File(h5_file, "r") as root:
    print(f"{('/entry/instrument/NDAttributes/SaveDest' in root)=}")

('/entry/instrument/NDAttributes/SaveDest' in root)=True


Configure our PV(s) for the *NeXus* addresses we want to use.  We **must**
reconnect our ophyd object after our change to the EPICS PVs to pick up this
change.  (The choices are only updated when the mbbo record is first connected.)

In [70]:
class FrameType(Device):
    zero = Component(EpicsSignal, ".ZRST", string=True)
    one = Component(EpicsSignal, ".ONST", string=True)
    two = Component(EpicsSignal, ".TWST", string=True)
    three = Component(EpicsSignal, ".THST", string=True)

for pv in ("FrameType", ):
    o = FrameType(f"{ad_prefix}cam1:{pv}", name="o")
    o.wait_for_connection()
    o.zero.put("/entry/data/data")
    o.one.put("/entry/data/dark")
    o.two.put("/entry/data/flat")
    o.three.put("")

# re-connect the detector object to pick up these different choices
adsimdet = MyDetector(ad_prefix, name='adsimdet')
set_ad_count_time(exposure=0.02, period=0.1)
ad_setup(det=adsimdet, nframes=NUM_FRAMES)
print(f"{adsimdet.cam.frame_type.enum_strs=}")

adsimdet.cam.frame_type.enum_strs=('/entry/data/data', '/entry/data/dark', '/entry/data/flat')


We also need to modify our layout file, adding groups,  datasets, and links in
the right places for the additional image types.  Here's the layout XML file
after those edits are complete:

In [71]:
xml_file = WORKSTATION_PREFIX / "layout.xml"  # IOC sees: /tmp/layout.xml
XML_LAYOUT2 = """
<?xml version="1.0" standalone="no" ?>
<hdf5_layout
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:noNamespaceSchemaLocation="../../../../../ADCore/XML_schema/hdf5_xml_layout_schema.xsd"
    >
  <global name="detector_data_destination" ndattribute="SaveDest" />
  <group name="entry">
    <attribute name="NX_class" source="constant" value="NXentry" type="string" />
    <attribute name="default" source="constant" value="data" type="string" />
    <group name="data">
        <attribute name="NX_class" source="constant" value="NXdata" type="string" />
        <attribute name="signal" source="constant" value="data" type="string" />
        <dataset name="data" source="detector">
          <attribute name="units" source="constant" value="counts" type="string" />
          <attribute name="description" source="constant" value="image frame(s)" type="string" />
          <attribute name="target" source="constant" value="/entry/data/data" type="string" />
        </dataset>
        <dataset name="dark" source="detector">
          <attribute name="units" source="constant" value="counts" type="string" />
          <attribute name="description" source="constant" value="dark (background) frame(s)" type="string" />
          <attribute name="target" source="constant" value="/entry/data/dark" type="string" />
        </dataset>
        <dataset name="flat" source="detector">
          <attribute name="units" source="constant" value="counts" type="string" />
          <attribute name="description" source="constant" value="flat (white) frame(s)" type="string" />
          <attribute name="target" source="constant" value="/entry/data/flat" type="string" />
        </dataset>
    </group>              <!-- end group data -->
    <group name="instrument">
      <attribute name="NX_class" source="constant" value="NXinstrument" type="string" />
      <group name="detector">
        <attribute name="NX_class" source="constant" value="NXdetector" type="string" />
        <hardlink name="data" target="/entry/data/data" />
        <hardlink name="dark" target="/entry/data/dark" />
        <hardlink name="flat" target="/entry/data/flat" />
      </group>            <!-- end group detector -->
      <group name="NDAttributes" ndattr_default="true">
        <attribute name="NX_class" source="constant" value="NXcollection" type="string" />
      </group>            <!-- end group NDAttribute (default) -->
    </group>              <!-- end group instrument -->
  </group>                <!-- end group entry -->
</hdf5_layout>
"""

with open(xml_file, "w") as f:
    f.write(XML_LAYOUT2.strip())
print(f"{xml_file.exists()=} {xml_file=}")
adsimdet.hdf1.layout_filename.put("/tmp/layout.xml")
print(f"{adsimdet.hdf1.layout_filename_valid.get()=}")

xml_file.exists()=True xml_file=PosixPath('/tmp/docker_ioc/iocad/tmp/layout.xml')
adsimdet.hdf1.layout_filename_valid.get()='Yes'


Collect a new image and check the output file.  This won't be a great test since
we are only writing one image type.  Just check that `SaveDest` is found and
that the expected data group is present.  All the test results should be `True`.

In [72]:
RE(bp.count([adsimdet]))

h5_file = local_h5_file(adsimdet)
with h5py.File(h5_file, "r") as NeXus_data:
    print(f"{('/entry/instrument/NDAttributes/SaveDest' in NeXus_data)=}")
    print(f"{('/entry/data/data' in NeXus_data)=}")
    frame_type = adsimdet.cam.frame_type  # short alias
    choices = frame_type.enum_strs
    choice = choices[frame_type.get()]
    print(f"{choice=}  {(choice in NeXus_data)=}")

('/entry/instrument/NDAttributes/SaveDest' in NeXus_data)=True
('/entry/data/data' in NeXus_data)=True
choice='/entry/data/data'  (choice in NeXus_data)=True


-------------------------------------

## FIX from this point

Image data not in the right places:

```
ds=<HDF5 dataset "dark": shape (15, 1024, 1024), type "|u1">  ds.shape=(15, 1024, 1024)
ds=<HDF5 dataset "data": shape (1, 1024, 1024), type "|u1">  ds.shape=(1, 1024, 1024)
ds=<HDF5 dataset "flat": shape (1, 1024, 1024), type "|u1">  ds.shape=(1, 1024, 1024)
```

## Series of images, darks, and flats

Process an arbitrary series.

In [81]:
import bluesky.plan_stubs as bps

IMAGE = 0
DARK = 1
FLAT = 2

def frame_set(det, frame_type=0, num_frames=1, sleep=0.25):
    frame_name = "image background white".split()[frame_type]
    print(f"{frame_type=}  {frame_name=}  {num_frames=}")
    yield from bps.mv(
        det.cam.frame_type, frame_type,
        det.cam.num_images, num_frames,
    )
    if sleep > 0:
        yield from bps.sleep(sleep)
    yield from bps.mv(det.cam.acquire, 1)  # waits for acquire=0
    while det.cam.acquire_busy.get(use_monitor=False) != 0:
        yield from bps.sleep(0.01)

def series(det, sequence, sleep=1):
    total = sum([item[1] for item in sequence])
    print("total frames:", total)

    print("setup")
    yield from bps.mv(
        det.hdf1.auto_save, "Yes",
        det.hdf1.num_capture, 0,
        det.cam.image_mode, "Multiple",
    )
    yield from bps.mv(det.hdf1.file_write_mode, 'Stream')  # TODO: or Capture
    yield from bps.stage(det)

    for frame_specification in sequence:
        frame_type, num_frames = frame_specification
        yield from frame_set(det, frame_type, num_frames, sleep=sleep)

    yield from bps.mv(det.hdf1.capture, 0)  # before file_write_mode = 'Single
    yield from bps.unstage(det)
    yield from bps.mv(
        det.cam.frame_type, IMAGE,
        det.hdf1.file_write_mode, 'Single',
    )

In [76]:
practice_device_staging(adsimdet)

Before staging 'adsimdet'
Device 'adsimdet' staged
Device 'adsimdet' unstaged


In [90]:
SEQUENCE = [
    (DARK, 1),
    (FLAT, 1),
    (IMAGE, 1),
    (IMAGE, 1),
    (DARK, 1),
    (IMAGE, 2),
    (FLAT, 1),
    (DARK, 1),
]

RE(series(adsimdet, SEQUENCE, sleep=0.0))

total frames: 9
setup
frame_type=1  frame_name='background'  num_frames=1
frame_type=2  frame_name='white'  num_frames=1
frame_type=0  frame_name='image'  num_frames=1
frame_type=0  frame_name='image'  num_frames=1
frame_type=1  frame_name='background'  num_frames=1
frame_type=0  frame_name='image'  num_frames=2
frame_type=2  frame_name='white'  num_frames=1
frame_type=1  frame_name='background'  num_frames=1


()

In [91]:
h5_file = local_h5_file(adsimdet)
print(f"{h5_file=}")
with h5py.File(h5_file, "r") as NeXus_data:
    print(f"{('/entry/data' in NeXus_data)=}")
    print(f"{('/entry/data/dark' in NeXus_data)=}")
    print(f"{('/entry/data/data' in NeXus_data)=}")
    print(f"{('/entry/data/flat' in NeXus_data)=}")
    print(f"{('/entry/instrument' in NeXus_data)=}")
    print(f"{('/entry/instrument/detector' in NeXus_data)=}")
    print(f"{('/entry/instrument/detector/dark' in NeXus_data)=}")
    print(f"{('/entry/instrument/detector/data' in NeXus_data)=}")
    print(f"{('/entry/instrument/detector/flat' in NeXus_data)=}")
    print(f"{('/entry/instrument/NDAttributes' in NeXus_data)=}")
    print(f"{('/entry/instrument/NDAttributes/SaveDest' in NeXus_data)=}")
    for addr in "dark data flat".split():
        try:
            ds = NeXus_data[f"/entry/data/{addr}"]
            print(f"{ds=}  {ds.shape=}")
        except KeyError:
            pass

h5_file=PosixPath('/tmp/docker_ioc/iocad/tmp/test_image_0032.h5')
('/entry/data' in NeXus_data)=True
('/entry/data/dark' in NeXus_data)=True
('/entry/data/data' in NeXus_data)=True
('/entry/data/flat' in NeXus_data)=True
('/entry/instrument' in NeXus_data)=True
('/entry/instrument/detector' in NeXus_data)=True
('/entry/instrument/detector/dark' in NeXus_data)=True
('/entry/instrument/detector/data' in NeXus_data)=True
('/entry/instrument/detector/flat' in NeXus_data)=True
('/entry/instrument/NDAttributes' in NeXus_data)=True
('/entry/instrument/NDAttributes/SaveDest' in NeXus_data)=True
ds=<HDF5 dataset "dark": shape (9, 1024, 1024), type "|u1">  ds.shape=(9, 1024, 1024)
ds=<HDF5 dataset "data": shape (1, 1024, 1024), type "|u1">  ds.shape=(1, 1024, 1024)
ds=<HDF5 dataset "flat": shape (1, 1024, 1024), type "|u1">  ds.shape=(1, 1024, 1024)


We are looking for this structure in the HDF5 data file (other structure has been omitted, for clarity):

```
  entry:NXentry
    data:NXdata
      dark:NX_UINT8[3,19,33] = [ ... ]
      data:NX_UINT8[4,19,33] = [ ... ]
      flat:NX_UINT8[2,19,33] = [ ... ]
    instrument:NXinstrument
      NDAttributes:NXcollection
        SaveDest:NX_CHAR[256] = /entry/data/data
      detector:NXdetector
        dark --> /entry/data/dark
        data --> /entry/data/data
        flat --> /entry/data/flat
```

All these tests should be `True`.